In [3]:
import pandas as pd, numpy as np
from get_draws.api import get_draws
from db_queries import get_population, get_ids

In [26]:
df = get_draws(gbd_id_type='rei_id',
                 gbd_id=240,
                 source='exposure',
                 #year_id=[2012,2020],
                 gbd_round_id=7,
                 status='best',
                 location_id = [179],
                 decomp_step = 'iterative')
df = (df.loc[(df.parameter.isin(['cat1','cat2']))
           &(df.age_group_id.isin([34,238,388,389]))]
      .groupby(['location_id','year_id','sex_id','age_group_id']).sum())
df = df.drop(columns=[c for c in df.columns if 'draw' not in c])
df.head()

draw_0    draw_1    draw_2  \
location_id year_id sex_id age_group_id                                 
179         1990    1      34            0.137402  0.145098  0.143457   
                           238           0.225488  0.217454  0.224907   
                           388           0.217272  0.230964  0.227420   
                           389           0.244524  0.246400  0.247079   
                    2      34            0.121842  0.119597  0.113912   

                                           draw_3    draw_4    draw_5  \
location_id year_id sex_id age_group_id                                 
179         1990    1      34            0.146232  0.137375  0.139113   
                           238           0.225610  0.227490  0.220548   
                           388           0.229223  0.225937  0.220480   
                           389           0.246134  0.233987  0.242240   
                    2      34            0.111966  0.121364  0.123997   

                                           draw_6    draw_7    draw_8  \
location_id year_id sex_id age_group_id                                 
179         1990    1      34            0.135576  0.145278  0.142828   
                           238           0.217154  0.220577  0.209508   
                           388           0.228384  0.227011  0.218963   
                           389           0.236660  0.242952  0.245419   
                    2      34            0.113350  0.115106  0.124347   

                                           draw_9  ...  draw_990  draw_991  \
location_id year_id sex_id age_group_id            ...                       
179         1990    1      34            0.138369  ...  0.140751  0.143343   
                           238           0.214134  ...  0.219538  0.214195   
                           388           0.224048  ...  0.230585  0.229853   
                           389           0.250417  ...  0.235449  0.236768   
                    2      34            0.115438  ...  0.119888  0.126152   

                                         draw_992  draw_993  draw_994  \
location_id year_id sex_id age_group_id                                 
179         1990    1      34            0.133295  0.140099  0.141069   
                           238           0.215534  0.219942  0.216440   
                           388           0.226507  0.233262  0.231427   
                           389           0.244913  0.254106  0.223391   
                    2      34            0.114219  0.118612  0.119304   

                                         draw_995  draw_996  draw_997  \
location_id year_id sex_id age_group_id                                 
179         1990    1      34            0.142206  0.145083  0.144043   
                           238           0.219449  0.223837  0.216999   
                           388           0.220181  0.217176  0.229410   
                           389           0.252184  0.237917  0.238762   
                    2      34            0.119973  0.117152  0.116238   

                                         draw_998  draw_999  
location_id year_id sex_id age_group_id                      
179         1990    1      34            0.139484  0.143266  
                           238           0.223291  0.220604  
                           388           0.222644  0.224971  
                           389           0.231669  0.236558  
                    2      34            0.119710  0.115465  

[5 rows x 1000 columns]

In [27]:
pop = get_population(gbd_round_id=7,
                    year_id=list(df.reset_index().year_id.unique()),
                    location_id=179,
                    sex_id=[1,2],
                    age_group_id=[34,238,388,389],
                    decomp_step='iterative')[['location_id','year_id','age_group_id','sex_id','population']]
pop

,location_id,year_id,age_group_id,sex_id,population
0,179,1990,34,1,2.806890e+06
1,179,1990,238,1,1.036745e+06
2,179,1990,388,1,5.004740e+05
3,179,1990,389,1,5.563624e+05
4,179,1990,34,2,2.683127e+06
...,...,...,...,...,...
259,179,2022,389,1,8.633414e+05
260,179,2022,34,2,4.636634e+06
261,179,2022,238,2,1.612204e+06
262,179,2022,388,2,7.132432e+05


In [28]:
data = df.reset_index().merge(pop, on=['age_group_id','sex_id','location_id','year_id'])
for i in list(range(0,1000)):
    data[f'draw_{i}'] = data[f'draw_{i}'] * data.population
data = data.groupby(['location_id','year_id']).sum()
for i in list(range(0,1000)):
    data[f'draw_{i}'] = data[f'draw_{i}'] / data.population
data = (data.drop(columns=[c for c in data.columns if 'draw' not in c])
        .apply(pd.DataFrame.describe, axis=1, percentiles=[0.025,0.975]))
data

count      mean       std       min      2.5%       50%  \
location_id year_id                                                             
179         1990     1000.0  0.161132  0.002052  0.153959  0.157037  0.161159   
            1991     1000.0  0.047544  0.001164  0.043837  0.045175  0.047542   
            1992     1000.0  0.047705  0.001167  0.044027  0.045361  0.047695   
            1993     1000.0  0.047513  0.001162  0.044050  0.045277  0.047489   
            1994     1000.0  0.046988  0.001151  0.043503  0.044787  0.046946   
            1995     1000.0  0.158217  0.002020  0.150753  0.154389  0.158196   
            1996     1000.0  0.044690  0.001110  0.040809  0.042700  0.044659   
            1997     1000.0  0.043461  0.001077  0.039514  0.041527  0.043445   
            1998     1000.0  0.042537  0.001050  0.038684  0.040621  0.042506   
            1999     1000.0  0.041654  0.001036  0.037777  0.039810  0.041604   
            2000     1000.0  0.147878  0.001922  0.141356  0.144229  0.147862   
            2001     1000.0  0.040023  0.001011  0.036288  0.038171  0.039997   
            2002     1000.0  0.039375  0.001008  0.035872  0.037525  0.039347   
            2003     1000.0  0.038749  0.000997  0.035266  0.036903  0.038716   
            2004     1000.0  0.037655  0.000980  0.034258  0.035795  0.037630   
            2005     1000.0  0.137425  0.001820  0.131779  0.133866  0.137355   
            2006     1000.0  0.034675  0.000909  0.031613  0.032953  0.034635   
            2007     1000.0  0.033075  0.000883  0.030064  0.031397  0.033050   
            2008     1000.0  0.031516  0.000845  0.028847  0.029843  0.031481   
            2009     1000.0  0.030081  0.000825  0.027298  0.028476  0.030037   
            2010     1000.0  0.120641  0.001660  0.114487  0.117296  0.120592   
            2011     1000.0  0.027594  0.000772  0.025118  0.026065  0.027567   
            2012     1000.0  0.026753  0.000763  0.024078  0.025302  0.026721   
            2013     1000.0  0.025848  0.000748  0.023317  0.024384  0.025814   
            2014     1000.0  0.024944  0.000738  0.022676  0.023512  0.024926   
            2015     1000.0  0.108137  0.001541  0.103511  0.105082  0.108130   
            2016     1000.0  0.022706  0.000699  0.020723  0.021348  0.022667   
            2017     1000.0  0.021497  0.000683  0.019677  0.020195  0.021492   
            2018     1000.0  0.020369  0.000670  0.018495  0.019077  0.020354   
            2019     1000.0  0.096375  0.001512  0.092027  0.093497  0.096303   
            2020     1000.0  0.093416  0.001484  0.089060  0.090663  0.093377   
            2021     1000.0  0.092076  0.001489  0.087748  0.089266  0.092030   
            2022     1000.0  0.091158  0.001493  0.086484  0.088232  0.091143   

                        97.5%       max  
location_id year_id                      
179         1990     0.165045  0.167857  
            1991     0.049940  0.051477  
            1992     0.050125  0.051732  
            1993     0.049921  0.051362  
            1994     0.049291  0.050662  
            1995     0.162113  0.164486  
            1996     0.046970  0.048451  
            1997     0.045753  0.046992  
            1998     0.044702  0.046244  
            1999     0.043836  0.045217  
            2000     0.151805  0.153603  
            2001     0.042065  0.043345  
            2002     0.041441  0.042789  
            2003     0.040774  0.042437  
            2004     0.039657  0.041629  
            2005     0.141088  0.144371  
            2006     0.036451  0.038426  
            2007     0.034834  0.036415  
            2008     0.033167  0.034780  
            2009     0.031704  0.033264  
            2010     0.123890  0.126993  
            2011     0.029093  0.030963  
            2012     0.028302  0.030008  
            2013     0.027386  0.028972  
            2014     0.026465  0.027976  
            2015     0.111235  0.113880  
          

In [30]:
data.reset_index()[['location_id','year_id','mean']].loc[data.reset_index().year_id<2010]

,location_id,year_id,mean
0,179,1990,0.161132
1,179,1991,0.047544
2,179,1992,0.047705
3,179,1993,0.047513
4,179,1994,0.046988
5,179,1995,0.158217
6,179,1996,0.044690
7,179,1997,0.043461
8,179,1998,0.042537
9,179,1999,0.041654


In [ ]:
# 9.1 (8.8 to 9.4)